# KNN

### Part3 - 移動回測

In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import os

In [64]:
data = pd.read_csv('new data/2022-06/train.csv')
data

,眾間,羅門遠東,氣管,神化,無涉標,死當然,看看分享,闖盪,近季,金臺企銀,...,監察人,益股,自來請益,效益台股,省稅,鋁價,東科華晶科德微中,豐田,label,post_time
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,2022-03-09
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-09
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-10
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-10
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2022-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-05-11
1221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-05-11
1222,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-05-11
1223,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2022-05-11


In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import os

# Define parameter grid
param_grid = {
    'n_neighbors': [11],
    'weights': ['uniform'],
    'leaf_size': [5],
    'p': [2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

# resampling strategies
resampling_strategies = ['under']

# Define thresholds
thresholds = [0.8]

models_list = []

# Initialize variables to store confusion matrix elements across all data files
total_predicted_rise_actual_rise = 0
total_predicted_rise_actual_fall = 0
total_predicted_fall_actual_rise = 0
total_predicted_fall_actual_fall = 0

total_true_trades = 0
total_trades = 0
total_days = 0

# Loop over parameter combinations
for n_neighbors in param_grid['n_neighbors']:
    for weights in param_grid['weights']:
        for leaf_size in param_grid['leaf_size']:
            for p in param_grid['p']:
                for strategy in resampling_strategies:
                    for threshold in thresholds:
                        print("Model Parameters:")
                        print("n_neighbors:", n_neighbors)
                        print("weights:", weights)
                        print("leaf_size:", leaf_size)
                        print("p:", p)
                        print("resampling strategy:", strategy)
                        print("threshold:", threshold)
                        
                        # Initialize variables to store confusion matrix elements & trade rate elements for the current model
                        model_predicted_rise_actual_rise = 0
                        model_predicted_rise_actual_fall = 0
                        model_predicted_fall_actual_rise = 0
                        model_predicted_fall_actual_fall = 0
                        
                        true_trades = 0
                        trades = 0
                        days = 0
                        
                        # Iterate over available data folders
                        data_folders = os.listdir('new data')  # Update path_to_data_folders with the directory containing your data folders         
                        for folder in data_folders:
                            folder_path = os.path.join('new data', folder)
                            if os.path.isdir(folder_path):
                                print("Processing folder:", folder)
                                    
                                # Load training data
                                train_data = pd.read_csv(os.path.join(folder_path, 'train.csv'))
                                
                                train_nonzero_indices = train_data['label'] != 0
                                train_data_nonzero = train_data[train_nonzero_indices]
                                
                                X_train = train_data_nonzero.drop(['label', 'post_time'], axis=1) 
                                y_train = train_data_nonzero['label']
                                
                                # Drop unnecessary columns
                                # X_train = train_data.drop(['label', 'post_time'], axis=1)
                                # y_train = train_data['label']

                                # Load testing data
                                test_data = pd.read_csv(os.path.join(folder_path, 'test.csv'))

                                test_nonzero_indices = test_data['label'] != 0
                                test_data_nonzero = test_data[test_nonzero_indices]
                                
                                X_test = test_data_nonzero.drop(['label', 'post_time'], axis=1) 
                                y_test = test_data_nonzero['label']
                                
                                # Drop unnecessary columns
                                # X_test = test_data.drop(['label', 'post_time'], axis=1)
                                # y_test = test_data['label']
                                    
                                model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, leaf_size=leaf_size, p=p)
                                
                                # Define resampling technique
                                if strategy == 'over':
                                    sampler = RandomOverSampler()
                                elif strategy == 'under':
                                    sampler = RandomUnderSampler() 
                                else:
                                    # No resampling
                                    X_resampled, y_resampled = X_train, y_train

                                # Apply resampling if applicable
                                if strategy in ['over', 'under']:
                                    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)        
                                                                
                                # Train the model
                                model.fit(X_resampled, y_resampled)
                                    
                                # Make predictions
                                y_pred = model.predict(X_test)
                                    
                                # Calculate accuracy
                                KNN_accuracy = accuracy_score(y_test, y_pred)
                                
                                # Add predicted labels to DataFrame
                                y_pred_df = pd.DataFrame(y_pred, columns=['predicted_label'], index=X_test.index)
                                y_test_df = pd.DataFrame(y_test, columns=['label'], index=X_test.index)
                                y_pred_df['post_time'] = test_data['post_time']
                                result_df = pd.concat([y_pred_df, y_test_df], axis=1)
                                
                                # Calculate predicted label counts
                                predicted_label_count = result_df.groupby('post_time')['predicted_label'].value_counts().unstack(fill_value=0)
                                
                                # Calculate actual label counts
                                actual_label_count = test_data.groupby('post_time')['label'].value_counts().unstack(fill_value=0)
                                merged_counts = pd.concat([predicted_label_count, actual_label_count], axis=1, keys=['Predicted', 'Actual']).dropna()
                                
                                # Calculate predicted and actual label expectations
                                predicted_expectation = (merged_counts['Predicted'][1] - merged_counts['Predicted'][-1]) / merged_counts['Predicted'].sum(axis=1)
                                actual_expectation = (merged_counts['Actual'][1] - merged_counts['Actual'][-1]) / merged_counts['Actual'].sum(axis=1)
                                
                                # Convert expectations to rise/fall labels
                                predicted_movement = pd.Series(['Rise' if exp > threshold else 'Fall' if exp < -threshold else 'X' for exp in predicted_expectation], index=merged_counts.index)
                                actual_movement = pd.Series(['Rise' if exp > threshold else 'Fall' if exp < -threshold else 'X' for exp in actual_expectation], index=merged_counts.index)
                                
                                # Copy DataFrame of actual labels and shift index by one day to represent t+1
                                actual_movement_t1 = actual_movement.shift(-1)
                                
                                # Merge predicted labels and t+1 actual labels
                                merged_movements = pd.concat([predicted_movement, actual_movement_t1], axis=1, keys=['predicted_movement', 'actual_movement'])
                                merged_movements = merged_movements.dropna()
                                
                                true_trades += (merged_movements['actual_movement'] != 'X').sum()
                                trades += (merged_movements['predicted_movement'] != 'X').sum()
                                days += len(merged_movements)
                                
                                # Calculate elements of confusion matrix
                                predicted_rise_actual_rise = ((predicted_movement == 'Rise') & (actual_movement == 'Rise')).sum()
                                predicted_rise_actual_fall = ((predicted_movement == 'Rise') & (actual_movement == 'Fall')).sum()
                                predicted_fall_actual_rise = ((predicted_movement == 'Fall') & (actual_movement == 'Rise')).sum()
                                predicted_fall_actual_fall = ((predicted_movement == 'Fall') & (actual_movement == 'Fall')).sum()

                                # Accumulate confusion matrix elements for the current model
                                model_predicted_rise_actual_rise += predicted_rise_actual_rise
                                model_predicted_rise_actual_fall += predicted_rise_actual_fall
                                model_predicted_fall_actual_rise += predicted_fall_actual_rise
                                model_predicted_fall_actual_fall += predicted_fall_actual_fall
                        
                        total_true_trades += true_trades
                        total_trades += trades
                        total_days += days
                                
                        # Accumulate confusion matrix elements across all data files for the current model
                        total_predicted_rise_actual_rise += model_predicted_rise_actual_rise
                        total_predicted_rise_actual_fall += model_predicted_rise_actual_fall
                        total_predicted_fall_actual_rise += model_predicted_fall_actual_rise
                        total_predicted_fall_actual_fall += model_predicted_fall_actual_fall
                        
                        # Put results into DataFrame
                        conf_matrix_df = pd.DataFrame({
                                'Predicted Rise': [total_predicted_rise_actual_rise, total_predicted_fall_actual_rise],
                                'Predicted Fall': [total_predicted_rise_actual_fall, total_predicted_fall_actual_fall]
                            }, index=['Actual Rise', 'Actual Fall'])

                        # Print results
                        print("Confusion Matrix: ")
                        print(conf_matrix_df)
                        
                        # Calculate accuracy
                        accuracy = (total_predicted_rise_actual_rise + total_predicted_fall_actual_fall) / (total_predicted_rise_actual_rise + total_predicted_rise_actual_fall + total_predicted_fall_actual_rise + total_predicted_fall_actual_fall)

                        # Print results
                        print()
                        print("Total Accuracy: ", accuracy)
                        
                        total_true_trade_rate = total_true_trades / total_days
                        total_trade_rate = total_trades / total_days
                        # print("True Active", total_true_trade_rate)
                        print("Active: ", total_trade_rate)
                            
                        models_list.append({
                                "n_neighbors": n_neighbors,
                                "weights": weights,
                                "leaf_size": leaf_size,
                                "p": p,
                                "resampling_strategy": strategy,
                                "threshhold": threshold,
                                "Accuracy": accuracy,
                                "Active": total_trade_rate,
                                "True Active": total_true_trade_rate
                            })
                            
                        if accuracy > 0.7 and total_trade_rate > 0.08:
                                print(" !!!! Good model !!!! ")
                                
                        print("==================================================")


Model Parameters:
n_neighbors: 11
weights: uniform
leaf_size: 5
p: 2
resampling strategy: under
threshold: 0.8
Processing folder: 2023-10
Processing folder: 2023-11
Processing folder: 2024-01
Processing folder: 2022-11
Processing folder: 2022-10
Processing folder: 2023-05
Processing folder: 2023-02
Processing folder: 2023-03
Processing folder: 2023-04
Processing folder: 2024-02
Processing folder: 2023-12
Processing folder: 2022-12
Processing folder: 2022-09
Processing folder: 2022-07
Processing folder: 2022-06
Processing folder: 2022-08
Processing folder: 2023-01
Processing folder: 2023-06
Processing folder: 2023-08
Processing folder: 2023-09
Processing folder: 2023-07
Confusion Matrix: 
             Predicted Rise  Predicted Fall
Actual Rise              22              17
Actual Fall              14              19

Total Accuracy:  0.5694444444444444
Active:  0.25390625


In [46]:
models_result = pd.DataFrame(models_list)
models_result

,n_neighbors,weights,leaf_size,p,resampling_strategy,threshhold,Accuracy,Active,True Active
0,11,uniform,5,2,under,0.8,0.595238,0.296875,1.0


In [47]:
print("準確率: ", accuracy) 
print("出手率: ", total_trade_rate) #Active
print()
conf_matrix_df = pd.DataFrame({
    '預測為漲': [total_predicted_rise_actual_rise, total_predicted_fall_actual_rise],
    '預測為下跌': [total_predicted_rise_actual_fall, total_predicted_fall_actual_fall]
}, index=['實際為漲', '實際為下跌'])


print("Confusion Matrix:")
conf_matrix_df

準確率:  0.5952380952380952
出手率:  0.296875

Confusion Matrix:


,預測為漲,預測為下跌
實際為漲,20,12
實際為下跌,22,30


-------------------------------------------

## Results

In [51]:
y_pred_series = pd.Series(y_pred)

value_counts = y_pred_series.value_counts()

print('預測label統計:')
print('# 1:', value_counts.get(1, 0))
print('# 0:', value_counts.get(0, 0))
print('# -1:', value_counts.get(-1, 0))

預測label統計:
# 1: 199
# 0: 0
# -1: 23


In [52]:
result_df

,predicted_label,post_time,label
0,1,2023-07-14,1
1,1,2023-07-14,1
2,1,2023-07-14,1
3,1,2023-07-14,1
4,1,2023-07-14,1
...,...,...,...
217,1,2023-07-19,-1
218,1,2023-07-21,-1
219,1,2023-07-21,-1
220,1,2023-07-21,-1


In [48]:
merged_counts

Predicted       Actual       
                  -1     1     -1  0   1
post_time                               
2023-07-01       2.0   6.0      0  0   8
2023-07-06       1.0  32.0     33  0   0
2023-07-09       0.0  14.0      0  0  14
2023-07-13       5.0  25.0      0  0  30
2023-07-14       3.0  29.0      0  0  32
2023-07-16       0.0   6.0      6  0   0
2023-07-19       0.0  12.0     12  0   0
2023-07-20       2.0   7.0      0  0   9
2023-07-21       3.0  20.0     23  0   0
2023-07-23       5.0  15.0      0  0  20
2023-07-27       2.0  24.0      0  0  26
2023-07-30       0.0   9.0      9  0   0

In [54]:
# 預測標籤和實際標籤期望值
predicted_expectation = (merged_counts['Predicted'][1] - merged_counts['Predicted'][-1]) / merged_counts['Predicted'].sum(axis=1)
actual_expectation = (merged_counts['Actual'][1] - merged_counts['Actual'][-1]) / merged_counts['Actual'].sum(axis=1)


expectation_df = pd.concat([predicted_expectation, actual_expectation], axis=1, keys=['Predicted Expectation', 'Actual Expectation'])

print("Expected Values: ")
expectation_df

Expected Values: 


,Predicted Expectation,Actual Expectation
post_time,,
2023-07-01,0.500000,1.0
2023-07-06,0.939394,-1.0
2023-07-09,1.000000,1.0
2023-07-13,0.666667,1.0
2023-07-14,0.812500,1.0
2023-07-16,1.000000,-1.0
2023-07-19,1.000000,-1.0
2023-07-20,0.555556,1.0
2023-07-21,0.739130,-1.0


In [55]:
# 交易行為
trading_actions = pd.DataFrame(index=merged_counts.index, columns=['Action'])


for date, pred_exp in predicted_expectation.items():
    if pred_exp > threshold:  # 買入
        trading_actions.loc[date, 'Action'] = 'Buy'
    elif pred_exp < -threshold:  # 賣出
        trading_actions.loc[date, 'Action'] = 'Sell'
    else:
        trading_actions.loc[date, 'Action'] = 'Hold'
    
        
trading_actions

,Action
post_time,
2023-07-01,Hold
2023-07-06,Buy
2023-07-09,Buy
2023-07-13,Hold
2023-07-14,Buy
2023-07-16,Buy
2023-07-19,Buy
2023-07-20,Hold
2023-07-21,Hold


In [56]:
# 預測label和實際label的期望值
predicted_expectation = (merged_counts['Predicted'][1] - merged_counts['Predicted'][-1]) / merged_counts['Predicted'].sum(axis=1)
actual_expectation = (merged_counts['Actual'][1] - merged_counts['Actual'][-1]) / merged_counts['Actual'].sum(axis=1)

# 轉換為漲跌標籤
predicted_movement = pd.Series(['Rise' if exp > threshold else 'Fall' if exp < -threshold else 'X' for exp in predicted_expectation], index=merged_counts.index)
actual_movement = pd.Series(['Rise' if exp > threshold else 'Fall' if exp < -threshold else 'X' for exp in actual_expectation], index=merged_counts.index)


# 合併預測label和實際label
merged_movements = pd.concat([predicted_movement, actual_movement], axis=1, keys=['predicted_movement', 'actual_movement'])
merged_movements = merged_movements.dropna()


merged_movements

,predicted_movement,actual_movement
post_time,,
2023-07-01,X,Rise
2023-07-06,Rise,Fall
2023-07-09,Rise,Rise
2023-07-13,X,Rise
2023-07-14,Rise,Rise
2023-07-16,Rise,Fall
2023-07-19,Rise,Fall
2023-07-20,X,Rise
2023-07-21,X,Fall


In [ ]:
# 預測為漲且實際為漲的樣本數
predicted_rise_actual_rise = ((predicted_movement == 'Rise') & (actual_movement == 'Rise')).sum()

# 預測為漲且實際為下跌的樣本數
predicted_rise_actual_fall = ((predicted_movement == 'Rise') & (actual_movement == 'Fall')).sum()

# 預測為下跌且實際為漲的樣本數
predicted_fall_actual_rise = ((predicted_movement == 'Fall') & (actual_movement == 'Rise')).sum()

# 預測為下跌且實際為下跌的樣本數
predicted_fall_actual_fall = ((predicted_movement == 'Fall') & (actual_movement == 'Fall')).sum()


conf_matrix_df = pd.DataFrame({
    '預測為漲': [predicted_rise_actual_rise, predicted_fall_actual_rise],
    '預測為下跌': [predicted_rise_actual_fall, predicted_fall_actual_fall]
}, index=['實際為漲', '實際為下跌'])


print("Confusion Matrix:")
conf_matrix_df

Confusion Matrix:


,預測為漲,預測為下跌
實際為漲,3,4
實際為下跌,0,0


#### Final Results

In [ ]:
print("準確率: ", accuracy) 
print("出手率: ", total_trade_rate) #Active
print()
conf_matrix_df = pd.DataFrame({
    '預測為漲': [total_predicted_rise_actual_rise, total_predicted_fall_actual_rise],
    '預測為下跌': [total_predicted_rise_actual_fall, total_predicted_fall_actual_fall]
}, index=['實際為漲', '實際為下跌'])


print("Confusion Matrix:")
conf_matrix_df

準確率:  0.5952380952380952
出手率:  0.296875

Confusion Matrix:


,預測為漲,預測為下跌
實際為漲,20,12
實際為下跌,22,30


--------------------